In [2]:
pip install PyPDF2 python-docx nltk

  Using cached nltk-3.9.2-py3-none-any.whl.metadata (3.2 kB)
Using cached nltk-3.9.2-py3-none-any.whl (1.5 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import PyPDF2

def extract_text_from_pdf(file):
    pdf = PyPDF2.PdfReader(file)
    text = ""
    for page in pdf.pages:
        text += page.extract_text()
    return text

In [3]:
import os
print(os.getcwd())

C:\Users\HP\Downloads\archive (23)


In [5]:
import os

base_path = r"C:\Users\HP\Downloads\archive (23)"

print(os.listdir(base_path))

['.ipynb_checkpoints', 'data', 'Resume', 'Untitled.ipynb']


In [6]:
import os
import PyPDF2

def extract_text_from_pdf(file_path):
    text = ""
    with open(file_path, "rb") as file:
        pdf = PyPDF2.PdfReader(file)
        for page in pdf.pages:
            if page.extract_text():
                text += page.extract_text()
    return text

folder_path = r"C:\Users\HP\Downloads\archive (23)\Resume"

resume_texts = []

for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".pdf"):
            full_path = os.path.join(root, file)
            print("Reading:", full_path)
            text = extract_text_from_pdf(full_path)
            resume_texts.append(text)

print("Total resumes:", len(resume_texts))

Total resumes: 0


In [7]:
import os

folder_path = r"C:\Users\HP\Downloads\archive (23)\Resume"

for root, dirs, files in os.walk(folder_path):
    for file in files:
        print(file)

Resume.csv


In [8]:
import pandas as pd

file_path = r"C:\Users\HP\Downloads\archive (23)\Resume\Resume.csv"

df = pd.read_csv(file_path)

print(df.head())

         ID                                         Resume_str  \
0  16852973           HR ADMINISTRATOR/MARKETING ASSOCIATE\...   
1  22323967           HR SPECIALIST, US HR OPERATIONS      ...   
2  33176873           HR DIRECTOR       Summary      Over 2...   
3  27018550           HR SPECIALIST       Summary    Dedica...   
4  17812897           HR MANAGER         Skill Highlights  ...   

                                         Resume_html Category  
0  <div class="fontsize fontface vmargins hmargin...       HR  
1  <div class="fontsize fontface vmargins hmargin...       HR  
2  <div class="fontsize fontface vmargins hmargin...       HR  
3  <div class="fontsize fontface vmargins hmargin...       HR  
4  <div class="fontsize fontface vmargins hmargin...       HR  


In [9]:
print(df.columns)

Index(['ID', 'Resume_str', 'Resume_html', 'Category'], dtype='object')


In [10]:
resume_texts = df['Resume_str'].tolist()

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english')

resume_vectors = vectorizer.fit_transform(resume_texts)

In [12]:
job_description = "Looking for data analyst with python sql and machine learning"

jd_vector = vectorizer.transform([job_description])

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

scores = cosine_similarity(resume_vectors, jd_vector)

In [14]:
df['Score'] = scores

df_sorted = df.sort_values(by='Score', ascending=False)

print(df_sorted[['Category', 'Score']].head(10))

                    Category     Score
1142              CONSULTANT  0.234070
1762             ENGINEERING  0.230622
1218              CONSULTANT  0.215311
563     BUSINESS-DEVELOPMENT  0.201155
1303           DIGITAL-MEDIA  0.194029
2153                 BANKING  0.192078
1339              AUTOMOBILE  0.179822
229   INFORMATION-TECHNOLOGY  0.176367
778               HEALTHCARE  0.175494
2385                AVIATION  0.161366


In [15]:
print(resume_texts[0][:200])

         HR ADMINISTRATOR/MARKETING ASSOCIATE

HR ADMINISTRATOR       Summary     Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management.   Resp


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(
    resume_vectors, df['Category'], test_size=0.2
)

model = LogisticRegression()
model.fit(X_train, y_train)

print("Accuracy:", model.score(X_test, y_test))

Accuracy: 0.6156941649899397


In [17]:
skills_list = [
    "python", "sql", "machine learning", "excel", 
    "power bi", "tableau", "java", "c++"
]

In [18]:
def extract_skills(text):
    found = []
    text = text.lower()
    
    for skill in skills_list:
        if skill in text:
            found.append(skill)
    
    return found

df['Skills'] = df['Resume_str'].apply(extract_skills)

In [19]:
job_description = "Looking for python sql and machine learning expert"

jd_skills = extract_skills(job_description)

def skill_score(resume_skills):
    if len(jd_skills) == 0:
        return 0
    match = len(set(resume_skills) & set(jd_skills))
    return match / len(jd_skills)

df['Skill_Score'] = df['Skills'].apply(skill_score)

In [20]:
df['Final_Score'] = (df['Score'] * 0.7) + (df['Skill_Score'] * 0.3)

df_sorted = df.sort_values(by='Final_Score', ascending=False)

In [21]:
print(df_sorted[['Category', 'Score', 'Skill_Score', 'Final_Score']].head(10))

                  Category     Score  Skill_Score  Final_Score
1762           ENGINEERING  0.230622     1.000000     0.461435
1218            CONSULTANT  0.215311     1.000000     0.450717
2153               BANKING  0.192078     1.000000     0.434455
1348            AUTOMOBILE  0.148966     1.000000     0.404276
563   BUSINESS-DEVELOPMENT  0.201155     0.666667     0.340809
1303         DIGITAL-MEDIA  0.194029     0.666667     0.335821
1339            AUTOMOBILE  0.179822     0.666667     0.325876
1717           ENGINEERING  0.033633     1.000000     0.323543
2385              AVIATION  0.161366     0.666667     0.312956
1756           ENGINEERING  0.151632     0.666667     0.306142


In [22]:
df_sorted.to_csv("ranked_candidates.csv", index=False)